# Sympy derivation of matrix elements

In [1]:
import sympy as sp

Define some symbols

In [2]:
theta_w, phi_t, PA, inc, azi = sp.symbols('theta_w, phi_t, PA_, inc_, azi_', real=True)
x, y, z = sp.symbols('x, y, z', real=True)

Define the rotation matrices.

In [3]:
def Rx(angle):
    return sp.Matrix([
    [1,       0,              0],
    [0,       sp.cos(angle), -sp.sin(angle)],
    [0,       sp.sin(angle),  sp.cos(angle)],
])

def Ry(angle):
    return sp.Matrix([
    [ sp.cos(angle), 0, sp.sin(angle)],
    [0,              1, 0],
    [-sp.sin(angle), 0, sp.cos(angle)],
])

def Rz(angle):
    return sp.Matrix([
    [sp.cos(angle), -sp.sin(angle), 0],
    [sp.sin(angle),  sp.cos(angle), 0],
    [0,              0,             1],
])

Define the initial coordinates $\vec p_0 = \begin{pmatrix}
x\\
y\\
z
\end{pmatrix}$

In [4]:
p0 = sp.Matrix([x, y, z])

We apply the tilting and twisting to the disk coordinates which is a rotation first around $\vec x$ and then around $\vec z$

In [67]:
p1 = Rz(phi_t) * Rx(theta_w) * p0

Next we get the observers view by projecting onto a new coordinate axis that is rotated by `PA`. If `PA` is zero, the observer is looking down onto the disk in negative z direction.

In [68]:
# unrotated base: top view
base0 = sp.Matrix([[1, 0, 0],[0, 1, 0], [0, 0, -1]])

# inclination of the entire disk
R_inc = Rx(inc)

# azimuthal rotation of the view
R_azi = Ry(azi)

# rotate the base
R_PA = Rz(PA)
base1 = R_PA * base0

now project onto that new base

In [69]:
p2 = ((R_inc * p1).T * base1).T

The result is rather messy ...

In [70]:
p2

Matrix([
[(-(y*sin(theta_w) + z*cos(theta_w))*sin(inc_) + (x*sin(phi_t) + y*cos(phi_t)*cos(theta_w) - z*sin(theta_w)*cos(phi_t))*cos(inc_))*sin(PA_) + (x*cos(phi_t) - y*sin(phi_t)*cos(theta_w) + z*sin(phi_t)*sin(theta_w))*cos(PA_)],
[(-(y*sin(theta_w) + z*cos(theta_w))*sin(inc_) + (x*sin(phi_t) + y*cos(phi_t)*cos(theta_w) - z*sin(theta_w)*cos(phi_t))*cos(inc_))*cos(PA_) - (x*cos(phi_t) - y*sin(phi_t)*cos(theta_w) + z*sin(phi_t)*sin(theta_w))*sin(PA_)],
[                                                                                             -(y*sin(theta_w) + z*cos(theta_w))*cos(inc_) - (x*sin(phi_t) + y*cos(phi_t)*cos(theta_w) - z*sin(theta_w)*cos(phi_t))*sin(inc_)]])

In [71]:
# Transformation matrix using Rich's definition inside eddy

def RRich(angle):
    return sp.Matrix([[sp.sin(angle),  sp.cos(angle), 0],
                      [sp.cos(angle), -sp.sin(angle), 0],
                      [0            ,  0            , 1]])

Rrich = RRich(PA)

pRich = Rrich * R_inc * p1

... so we rather let `sympy` write the fortran code for us. This is what is put into the `fortran.f90` file.

In [72]:
from sympy.printing import fcode
print(fcode(p2[0], standard=2003, source_format='free'))
print('')
print(fcode(p2[1], standard=2003, source_format='free'))
print('')
print(fcode(p2[2], standard=2003, source_format='free'))

(-(y*sin(theta_w) + z*cos(theta_w))*sin(inc_) + (x*sin(phi_t) + y*cos( &
      phi_t)*cos(theta_w) - z*sin(theta_w)*cos(phi_t))*cos(inc_))*sin( &
      PA_) + (x*cos(phi_t) - y*sin(phi_t)*cos(theta_w) + z*sin(phi_t)* &
      sin(theta_w))*cos(PA_)

(-(y*sin(theta_w) + z*cos(theta_w))*sin(inc_) + (x*sin(phi_t) + y*cos( &
      phi_t)*cos(theta_w) - z*sin(theta_w)*cos(phi_t))*cos(inc_))*cos( &
      PA_) - (x*cos(phi_t) - y*sin(phi_t)*cos(theta_w) + z*sin(phi_t)* &
      sin(theta_w))*sin(PA_)

-(y*sin(theta_w) + z*cos(theta_w))*cos(inc_) - (x*sin(phi_t) + y*cos( &
      phi_t)*cos(theta_w) - z*sin(theta_w)*cos(phi_t))*sin(inc_)


In [73]:
print(fcode(pRich[0], standard=2003, source_format='free'))
print('')
print(fcode(pRich[1], standard=2003, source_format='free'))
print('')
print(fcode(pRich[2], standard=2003, source_format='free'))

-(y*sin(theta_w) + z*cos(theta_w))*sin(inc_)*cos(PA_) + (x*sin(phi_t) + &
      y*cos(phi_t)*cos(theta_w) - z*sin(theta_w)*cos(phi_t))*cos(PA_)* &
      cos(inc_) + (x*cos(phi_t) - y*sin(phi_t)*cos(theta_w) + z*sin( &
      phi_t)*sin(theta_w))*sin(PA_)

(y*sin(theta_w) + z*cos(theta_w))*sin(PA_)*sin(inc_) - (x*sin(phi_t) + y &
      *cos(phi_t)*cos(theta_w) - z*sin(theta_w)*cos(phi_t))*sin(PA_)* &
      cos(inc_) + (x*cos(phi_t) - y*sin(phi_t)*cos(theta_w) + z*sin( &
      phi_t)*sin(theta_w))*cos(PA_)

(y*sin(theta_w) + z*cos(theta_w))*cos(inc_) + (x*sin(phi_t) + y*cos( &
      phi_t)*cos(theta_w) - z*sin(theta_w)*cos(phi_t))*sin(inc_)
